<a href="https://colab.research.google.com/github/Deep-Learning-Challenge/challenge-notebooks/blob/master/1.Multilayer%20Perceptrons/1.Lessons/1.Your%20First%20Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

# Your First Neural Network

Keras is a powerful and easy-to-use Python library for developing and evaluating deep learning models. It wraps the efficient numerical computation libraries Theano and TensorFlow and allows you to define and train neural network models in a few short lines of code. 

In this lesson, you will discover how to create your first neural network model in Python using Keras. After completing this lesson, you will know:

* How to load a CSV dataset ready for use with Keras.
* How to define and compile a Multilayer Perceptron model in Keras.
* How to evaluate a Keras model on a validation dataset.

Let's get started.

## Lesson Overview

There is not much code required, but we will slowly step over it to create your future models. The steps you are going to cover in this tutorial are as follows:

* Load Data.
* Define Model.
* Compile Model.
* Fit Model.
* Evaluate Model.
* Tie It All Together.

## Pima Indians Onset of Diabetes Dataset

This tutorial will use the Pima Indians onset of diabetes dataset, a standard machine learning dataset available for free download from the UCI Machine Learning repository. It describes patient medical record data for Pima Indians and whether they had an onset of diabetes within five years. It is a binary classification problem (onset of diabetes as 1 or not as 0). The input variables that describe each patient are numerical and have varying scales. 

Below lists the eight attributes for the dataset:

1. The number of times pregnant.
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
3. Diastolic blood pressure (mm Hg).
4. Triceps skinfold thickness (mm).
5. 2-Hour serum insulin (mu U/ml).
6. Body mass index.
7. Diabetes pedigree function.
8. Age (years).
9. Class, the onset of diabetes within five years.

Given that all attributes are numerical makes it easy to use directly with neural networks that expect numerical inputs and output values, and ideal for our first neural network in Keras. This dataset will also be used for a number of additional lessons coming up in this book, so keep it handy. Below is a sample of the dataset showing the first 5 rows of the 768 instances:

```
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
```

The baseline accuracy if all predictions are made as *no onset of diabetes* is `65.1%`. Top results on the dataset are in the range of `77.7%` accuracy using 10-fold cross-validation.

## Runtime Setup

In [2]:
import sys

dataset_name = "pima-indians-diabetes.data.csv"
if 'google.colab' in sys.modules:
    DATASET = f"https://github.com/Deep-Learning-Challenge/challenge-notebooks/raw/master/datasets/{dataset_name}"
else:
    DATASET = f"../../datasets/{dataset_name}"
    
DATASET

'../../datasets/pima-indians-diabetes.data.csv'

## Load Data

Whenever we work with machine learning algorithms that use a stochastic process (e.g., random numbers), it is a good idea to initialize the random number generator with a fixed seed value. This is so that you can run the same code again and again and get the same result. This is useful if you need to demonstrate a result, compare algorithms using the exact source of randomness or to debug a part of your code. You can initialize the random number generator with any seed
you like, for example:

In [3]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy

# fix random seed for reproducibility
numpy.random.seed(7)

2021-10-04 21:23:27.778446: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-04 21:23:27.778547: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Now we can load our Pima Indians dataset. You can now load the file directly using the NumPy function `loadtxt()`. There are eight input variables and one output variable (the last column). Once loaded, we can split the dataset into input variables (X) and the output class variable (Y ).

In [4]:
# load pima indians dataset
dataset = numpy.loadtxt(DATASET, delimiter=",")

# split into input and output variables
X = dataset[:,0:8]
Y = dataset[:,8]

We have initialized our random number generator to ensure our results are reproducible and loaded our data. We are now ready to define our neural network model.

## Define Model

Models in Keras are defined as a sequence of layers. We create a **Sequential** model and add layers one at a time until we are happy with our network topology. The first thing to get right is to ensure the input layer has the correct number of inputs. This can be specified when creating the first layer with the `input_dim` argument and set it to 8 for the eight input variables.

How do we know the number of layers to use and their types? This is a tough question. We can use heuristics, and often the best network structure is found through trial and error experimentation. Generally, you need a network large enough to capture the problem's structure, if that helps at all. In this example, we will use a fully-connected network structure with three layers.

Fully connected layers are defined using the `Dense` class. We can specify the number of neurons in the layer as the first argument and specify the activation function using the `activation` argument. We will use the rectifier (`relu`) activation function on the first two layers and the `sigmoid` activation function in the output layer. It used to be the case that sigmoid and `tanh` activation functions were preferred for all layers. These days, better performance is seen using the rectifier activation function. We use a sigmoid activation function on the output layer to ensure our network output is between 0 and 1 and easy to map to either a probability of class 1 or snap to a hard classification of either class with a default threshold of 0.5. We can piece it all together by adding each layer. The first hidden layer has 12 neurons and expects 8 input variables (e.g. `input_dim=8`). The second hidden layer has eight neurons and finally, the output layer has one neuron to predict the class (onset of diabetes or not).

In [5]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

2021-10-04 21:23:30.545463: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-04 21:23:30.545527: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-04 21:23:30.545554: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (0c5a5437e2e9): /proc/driver/nvidia/version does not exist
2021-10-04 21:23:30.546133: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


The figure below provides a depiction of the network structure.

![Network structure](../../images/network_structure.png)

## Compile Model

Now that the model is defined, we can compile it. Compiling the model uses efficient numerical libraries under the covers (the so-called backend) such as Theano or TensorFlow. The backend automatically chooses the best way to represent the network for training and making predictions to run on your hardware. When compiling, we must specify some additional properties required when training the network. Remember training a network means finding the best set of weights to make predictions for this problem.

We must specify the loss function to evaluate a set of weights, the optimizer used to search through different weights for the network, and any optional metrics we would like to collect and report during training. In this case, we will use logarithmic loss, which for a binary classification problem is defined in Keras as `binary_crossentropy`. We will also use the efficient gradient descent algorithm `adam` for no other reason than it is an efficient default. Learn more about the Adam optimization algorithm in the paper [Adam: A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980). Finally, we will collect and report the classification `accuracy` as the metric because it is a classification problem.

In [6]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fit Model

We have defined our model and compiled it ready for efficient computation. Now it is time to execute the model on some data. We can train or fit our model on our loaded data by calling the `fit()` function on the model.

The training process will run for a fixed number of iterations through the dataset called epochs, which we must specify using the `epochs` argument. We can also set the number of evaluated instances before a weight update in the network is performed, called the batch size, and set using the `batch_size` argument. For this problem, we will run for a small number of epochs (150) and use a relatively small batch size of 10. Again, these can be chosen experimentally by trial and error.

This is where the work happens on your CPU or GPU.

In [7]:
# Fit the model
model.fit(X, Y, epochs=150, batch_size=10)

2021-10-04 21:23:30.794058: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/150
77/77 [==============================] - 0s 1ms/step - loss: 2.6529 - accuracy: 0.4544
Epoch 2/150
77/77 [==============================] - 0s 1ms/step - loss: 1.3909 - accuracy: 0.5000
Epoch 3/150
77/77 [==============================] - 0s 1ms/step - loss: 0.9635 - accuracy: 0.5677
Epoch 4/150
77/77 [==============================] - 0s 1ms/step - loss: 0.8244 - accuracy: 0.5885
Epoch 5/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7495 - accuracy: 0.5938
Epoch 6/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7465 - accuracy: 0.6185
Epoch 7/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7134 - accuracy: 0.6289
Epoch 8/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6734 - accuracy: 0.6224
Epoch 9/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6735 - accuracy: 0.6497
Epoch 10/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6525 - accuracy: 0.6719
Epoch 11/

## Evaluate Model

We have trained our neural network on the entire dataset, and we can evaluate the network's performance on the same dataset. This will only give us an idea of how well we have modeled the dataset (e.g., train accuracy), but no idea of how well the algorithm might perform on new data. We have done this for simplicity, but ideally, you could separate your data into train and test datasets for the training and evaluation of your model.

You can evaluate your model on your training dataset using the `evaluation()` function on your model and pass it the same input and output used to train the model. This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

In [8]:
# evaluate the model
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

24/24 [==============================] - 0s 1ms/step - loss: 0.4875 - accuracy: 0.7539

accuracy: 75.39%


## Summary

In this lesson, you discovered how to create your first neural network model using the powerful Keras Python library for deep learning. Specifically, you learned the five key steps in using Keras to create a neural network or deep learning model, step-by-step including:

* How to load data.
* How to define a neural network model in Keras.
* How to compile a Keras model using efficient numerical backend.
* How to train a model on data.
* How to evaluate a model on data.